# Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import progressbar

# Grabbing URL

In [2]:
allPlayersURL = "https://www.basketball-reference.com/players/a"
allPlayersPageRequest = requests.get(allPlayersURL)

## Parse URL Page

In [3]:
page_soup = BeautifulSoup(allPlayersPageRequest.content, "html.parser")
table = page_soup.find("table")


## Creating Player Class

In [4]:
# class Player:
#     def __init__(self, name, college, active_from, active_to, position, url):
#         self.name = name    # instance variable unique to each instance
#         self.college = college
#         self.active_from = active_from,
#         self.active_to = active_to,
#         self.position = position,
#         self.url = url

## Grabbing NBA Data

In [4]:
count = 0
playersDB = []
if table:
    table_body = table.find("tbody")
    for row in table_body:
        table_body.find_all("tr")
        player_url = row.find("a")
        try:
            # all data for all players uniform across database
            cells = row.findAll("td")
            active_from = int(cells[0].text)
        except (AttributeError):
            continue
        if (active_from > 2009):
            try:
                count += 1
                player_names = player_url.text
                active_to = int(cells[1].text)
                position = cells[2].text
                college = cells[6].text
                player_entry = {
                    'name': player_names,
                    'active_from': active_from,
                    'active_to': active_to,
                    'position': position,
                    'college': college,
                    'url': "https://www.basketball-reference.com"+player_url['href']
                }
                playersDB.append(player_entry)
            except (AttributeError):
                continue

## Grabbing College Data

In [15]:
allCollegePlayersURL = "https://www.sports-reference.com/cbb/players/a-index.html"
allCollegePlayersPageRequest = requests.get(allCollegePlayersURL)
college_page_soup = BeautifulSoup(allCollegePlayersPageRequest.content, "html.parser")

college_div = college_page_soup.find("div", id="content")
college_list_header = college_div.find_next("h2").find_next("p")
college_total_list = college_list_header.find_next_siblings("div")

# targets = college_page_soup.find_all('small')
# for i in targets:
#     try:
#         i.select_one('a').decompose()
#     except AttributeError:
#         continue

college_duration = ""
playersDBWithCollege = []
# bar = progressbar.ProgressBar(max_value=len(playersDB))
giant_list = []
print(len(college_total_list))
for j in college_total_list:
    short_list = j.find_all_next("p")
    print(short_list)

for i in playersDB:
    # count += 1
    # if (i["college"] != ""):
    #     for j in college_total_list:
    #         try:
    #             if(i["name"] == j.find("a").text):
    #                 print("NAME: "+j.find("a").text)
    #                 college_duration = j.find("small").text
    #                 print("YEAR: "+j.find("small").text)
    #         except AttributeError:
    #             continue
    # player_entry_college = {
    #     'name': i["name"],
    #     'active_from': i["active_from"],
    #     'active_to': i["active_to"],
    #     'position': i["position"],
    #     'college': i["college"],
    #     'college_duration': college_duration,
    #     'url': i["url"]
    # }
    # playersDBWithCollege.append(player_entry_college)
    college_duration = ""
    # bar.update(count)

In [ ]:
# bar = progressbar.ProgressBar(max_value=len(playersDB))
# count = 0
# playersDBWithCollege = []
# college_start_year = ""
# college_end_year = ""
# for i in playersDB:
#     count += 1
#     if (i["college"] != ""):
#         playerPageRequest = requests.get(i["url"])
#         html_doc = playerPageRequest.text.replace('<!--', '').replace('-->', '')
#         playerPageSoup = BeautifulSoup(html_doc, "html.parser")
#         college_div = playerPageSoup.find("div", id="all_all_college_stats")
#         college_table = college_div.find("table")
#         college_stats = college_table.find("tbody")
#         college_rows = college_stats.find_all("tr")
#         college_years_played = []
#         for college_year in college_rows:
#             season = college_year.find("th")
#             college_years_played.append(season.text)
#         college_start_year = college_years_played[0]
#         if (college_years_played[0] != college_years_played[-1]):
#             college_end_year = college_years_played[-1]
#         else:
#             college_end_year = ""
#     print(i["name"])
#     print(college_start_year)
#     print(college_end_year)
#     player_entry_college = {
#         'name': i["name"],
#         'active_from': i["active_from"],
#         'active_to': i["active_to"],
#         'position': i["position"],
#         'college': i["college"],
#         'college_start_year': college_start_year,
#         'college_end_year': college_end_year,
#         'url': i["url"]
#     }
#     playersDBWithCollege.append(player_entry_college)
#     college_start_year = ""
#     college_end_year = ""
#     bar.update(count)

## Create DF

In [8]:
df = pd.DataFrame(playersDBWithCollege)

## Output df to csv

In [9]:
df.to_csv("playerData.csv", encoding="utf-8", index_label="index")